<div style="border:solid green 2px; padding: 20px">
    
# Отток клиентов


Из некоторого банка стали уходить клиенты. 

Маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

**Цель исследования:** нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.


### Описание данных

**Признаки**
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — количество недвижимости у клиента
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

**Целевой признак**
- Exited — факт ухода клиента
    
</div>

## Этапы исследования

[Шаг 1. Подготовка данных](#1)

[Шаг 2. Исследование задачи](#2)

[Шаг 3. Борьба с дисбалансом](#3)

[Шаг 4. Тестирование модели](#4)

[Шаг 5. Общий вывод](#5)

<a id='1'></a>

# 1. Подготовка данных

In [1]:
import pandas as pd

data = pd.read_csv('/datasets/Churn.csv')

# Выведем общую информацию по датасету
display(data.info())
display(data.describe())
data.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


В файле не достает данных в столбце о количестве недвижимости у клиента. Можно предположить, что пропущенные значения - это ее отсутствие. Заменим пропуски на "0" и переведем в целочисленный вид.

Присутсвуют столбцы, на наш взгляд, сбивающие адекватное обучение модели. Это "RowNumber" — индекс строки в данных и "CustomerId" — уникальный идентификатор клиента. Все это может заменить индекс строк, прописывающихся в Питоне. Удалим их.

Также проверим датасет на наличие дубликатов.

Булевы значения в столбцах "HasCrCard",	"IsActiveMembe" и "Exited" оставим пока в интовском типе.

In [2]:
data['Tenure'] = data['Tenure'].fillna('0').astype('int')

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)

# Проверим на дубликаты 
data.duplicated().sum()
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


**Выделим целевой признак и разобьем данные на 3 выборки: обучающую, валидационную и тестовую.**

Но перед этим проведем прямое кодирование (будем использовать OHE, так как оно работает со всеми моделями), поскольку перед нами задача бинарной классификации.

In [3]:
# Проведем прямое кодирование
data = pd.get_dummies(data, drop_first = True)

# Выделим целевой признак
features = data.drop('Exited', axis = 1)
target = data['Exited']


# Разобьем на 3 выборки
from sklearn.model_selection import train_test_split

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              stratify = target,
                                                                              test_size = 0.2,
                                                                              random_state = 12345)
features_train, features_test, target_train, target_test = train_test_split(features_train, 
                                                                              target_train,
                                                                              stratify = target_train,
                                                                              test_size = 0.25,
                                                                              random_state = 12345)

display(features_train.shape)
display(features_valid.shape)
features_test.shape

(6000, 11)

(2000, 11)

(2000, 11)

Разбили в 2 этапа:
1. Первоначально разделили исходные данные на обучающую и валидационную выборки в соотношении 4:1.
2. Вторым этапом разделили обучающую выборку на новую обучающую и тестовую выборки в соотношении 3:1.

Как итог, получили 3 датасета в соотношении 3:1:1 (60%-20%-20%)

**Стандартизируем данные, методом масштабирования признаков**

In [4]:
# Стандартизируем данные
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember','EstimatedSalary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
# Преобразим обучающую, валидационную и тестовую выборки
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

**Итак, данные готовы. Можно обучаться.**

In [5]:
data.corr()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
CreditScore,1.000000,-0.003965,0.003087,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094,0.005538,0.004780,-0.002857
Age,-0.003965,1.000000,-0.007368,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323,0.046897,-0.001685,-0.027544
Tenure,0.003087,-0.007368,1.000000,-0.005821,0.010106,0.021387,-0.025856,0.011225,-0.013319,0.001676,-0.000745,0.014889
Balance,0.006268,0.028308,-0.005821,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533,0.401110,-0.134892,0.012087
NumOfProducts,0.012238,-0.030680,0.010106,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820,-0.010419,0.009039,-0.021859
HasCrCard,-0.005458,-0.011721,0.021387,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138,0.010577,-0.013480,0.005766
IsActiveMember,0.025651,0.085472,-0.025856,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128,-0.020486,0.016732,0.022544
EstimatedSalary,-0.001384,-0.007201,0.011225,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097,0.010297,-0.006482,-0.008112
Exited,-0.027094,0.285323,-0.013319,0.118533,-0.047820,-0.007138,-0.156128,0.012097,1.000000,0.173488,-0.052667,-0.106512
Geography_Germany,0.005538,0.046897,0.001676,0.401110,-0.010419,0.010577,-0.020486,0.010297,0.173488,1.000000,-0.332084,-0.024628


<a id='2'></a>

# 2. Исследование задачи

Посмотрим на частоту встречаемости классов целевого признака.

In [6]:
class_frequency = pd.Series(target_train).value_counts(normalize = 2 / len(pd.Series(target_train)))
class_frequency

0    0.796167
1    0.203833
Name: Exited, dtype: float64

20% положительного класса и 80% отрицательного. Наблюдается дисбаланс.

Для начала проверим обучаемость модели без изменений баланса.<br>
Перед этим напишем функцию обучения модели, счета метрики F1 и AUC-ROC.

In [7]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

def obuchenie(model, features, target):
    model.fit(features, target)
    predictid_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predictid_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    return f1, auc_roc

### 2.1 Логистическая регрессия

In [8]:
from sklearn.linear_model import LogisticRegression

model_1 = LogisticRegression(solver = 'liblinear', random_state = 12345)
f1, auc_roc = obuchenie(model_1, features_train, target_train)

print('F1:', f1, '| AUC-ROC:', auc_roc)

F1: 0.29906542056074764 | AUC-ROC: 0.774481723634266


### 2.2 Решающее дерево

In [9]:
from sklearn.tree import DecisionTreeClassifier

model_2 = DecisionTreeClassifier(random_state = 12345)
f1, auc_roc = obuchenie(model_2, features_train, target_train)

print('F1:', f1, '| AUC-ROC:', auc_roc)

F1: 0.4945188794153471 | AUC-ROC: 0.6831585051924035


### 2.3 Случайный лес

In [10]:
from sklearn.ensemble import RandomForestClassifier

model_3 = RandomForestClassifier(random_state = 12345)
f1, auc_roc = obuchenie(model_3, features_train, target_train)

print('F1:', f1, '| AUC-ROC:', auc_roc)

F1: 0.5437500000000001 | AUC-ROC: 0.8239472137777222


Делаем вывод, что прогноз класса "1" не удался (F1-мера слишком далека от нужного значения), даже не смотря на то, что достигли хороших результатов в метрике AUC-ROC.<br>
Нужно побороть дисбаланс.

<a id='3'></a>

# 3. Борьба с дисбалансом

In [11]:
# Логистическая регрессия
def regressia(features, target):
    print('Логистическая регрессия')
    model_1 = LogisticRegression(random_state=12345, solver='liblinear')
    f1, auc_roc = obuchenie(model_1, features, target)
    print("F1:", f1, '| AUC-ROC:', auc_roc)
    print()


# Решающее дерево
def derevo(features, target):
    best_depth = 0
    best_f1 = 0
    best_auc_roc = 0
    best_model_2 = None
    for depth in range(1, 20):
        model_2 = DecisionTreeClassifier(max_depth = depth, random_state = 12345, class_weight = 'balanced')
        f1, auc_roc = obuchenie(model_2, features, target)
        if f1 > best_f1:
            best_f1 = f1
            best_depth = depth
            best_model_2 = model_2
            best_auc_roc = auc_roc
    print('Решающее дерево')
    print('Оптимальная глубина:', best_depth)
    print('F1:', best_f1, '| AUC-ROC:', best_auc_roc)
    print()


# Случайный лес
def lec(features, target):   
    best_est = 0
    best_depth = 0
    best_f1 = 0
    best_auc_roc = 0
    best_model_3 = None
    for est in range(10, 101, 10):
        for depth in range(1, 13):
            model_3 = RandomForestClassifier(random_state = 12345, n_estimators = est, max_depth=depth, class_weight = 'balanced')
            f1, auc_roc = obuchenie(model_3, features, target)
            if f1 > best_f1:
                best_f1 = f1
                best_est = est
                best_depth = depth
                best_model_3 = model_3
                best_auc_roc = auc_roc
    print('Случайный лес')
    print('Оптимальное количество деревьев:', best_est)
    print('Оптимальная глубина:', best_depth)
    print('F1:', best_f1, '| AUC-ROC:', best_auc_roc)
    print()

Для каждой модели (с ее лучшими гиперпараметрами) проведем работу над дисбалансом разными подходами.

Параллельно вычислим метрику F1 и сравним ее с AUC-ROC.

Подходы:
1. Взвешивание классов
2. Увеличение выборки нечастого класса
3. Уменьшение выборки частого класса
4. Изменение порога

### 3.1 Взвешивание классов

In [12]:
%%time

print('Логистическая регрессия')
model_1 = LogisticRegression(random_state=12345, solver='liblinear', class_weight = 'balanced')
f1, auc_roc = obuchenie(model_1, features_train, target_train)
print("F1:", f1, '| AUC-ROC:', auc_roc)
print()



print('Решающее дерево')
best_depth = 0
best_f1 = 0
best_auc_roc = 0
best_model_2 = None
for depth in range(1, 20):
    model_2 = DecisionTreeClassifier(max_depth = depth, random_state = 12345, class_weight = 'balanced')
    f1, auc_roc = obuchenie(model_2, features_train, target_train)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth
        best_model_2 = model_2
        best_auc_roc = auc_roc
print('Оптимальная глубина:', best_depth)
print('F1:', best_f1, '| AUC-ROC:', best_auc_roc)
print()



print('Случайный лес')
best_est = 0
best_depth = 0
best_f1 = 0
best_auc_roc = 0
best_model_3 = None
for est in range(10, 101, 10):
    for depth in range(1, 13):
        model_3 = RandomForestClassifier(random_state = 12345, n_estimators = est, max_depth=depth, class_weight = 'balanced')
        f1, auc_roc = obuchenie(model_3, features_train, target_train)
        if f1 > best_f1:
            best_f1 = f1
            best_est = est
            best_depth = depth
            best_model_3 = model_3
            best_auc_roc = auc_roc
print('Оптимальное количество деревьев:', best_est)
print('Оптимальная глубина:', best_depth)
print('F1:', best_f1, '| AUC-ROC:', best_auc_roc)
print()

Логистическая регрессия
F1: 0.5055319148936169 | AUC-ROC: 0.7811201031540015

Решающее дерево
Оптимальная глубина: 6
F1: 0.5711727842435094 | AUC-ROC: 0.8363810651946244

Случайный лес
Оптимальное количество деревьев: 60
Оптимальная глубина: 7
F1: 0.6436285097192225 | AUC-ROC: 0.8665661038542395

CPU times: user 46.4 s, sys: 270 ms, total: 46.7 s
Wall time: 46.8 s


### 3.2 Увеличение выборки нечастого класса

In [13]:
%%time

from sklearn.utils import shuffle

# Для этого напишем функцию с применением техники "upsampling"

def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)


# Логистическая регрессия
regressia(features_upsampled, target_upsampled)

# Решающее дерево
derevo(features_upsampled, target_upsampled)
    
# Случайный лес
lec(features_upsampled, target_upsampled) 

Логистическая регрессия
F1: 0.5067114093959731 | AUC-ROC: 0.781203391372883

Решающее дерево
Оптимальная глубина: 6
F1: 0.5711727842435094 | AUC-ROC: 0.8363810651946244

Случайный лес
Оптимальное количество деревьев: 80
Оптимальная глубина: 9
F1: 0.6343519494204426 | AUC-ROC: 0.8672617147193419

CPU times: user 1min 4s, sys: 395 ms, total: 1min 5s
Wall time: 1min 5s


### 3.3 Уменьшение выборки частого класса

In [14]:
%%time

# Для этого также напишем функцию, но на этот раз с применением техники "downsampling"

def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)



# Логистическая регрессия
regressia(features_downsampled, target_downsampled)

# Решающее дерево
derevo(features_downsampled, target_downsampled)
    
# Случайный лес
lec(features_downsampled, target_downsampled) 

Логистическая регрессия
F1: 0.5046102263202011 | AUC-ROC: 0.7796934068120509

Решающее дерево
Оптимальная глубина: 6
F1: 0.5852156057494866 | AUC-ROC: 0.8337119862543592

Случайный лес
Оптимальное количество деревьев: 30
Оптимальная глубина: 7
F1: 0.6276067527308838 | AUC-ROC: 0.86794730015069

CPU times: user 28.4 s, sys: 236 ms, total: 28.6 s
Wall time: 28.8 s


### 3.4 Изменение порога

In [15]:
%%time

import numpy as np

# Для этого напишем функцию
def porog(model):
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    best_f1 = 0
    best_threshold = 0
    for threshold in np.arange(0, 0.5, 0.02):
        predicted_valid = probabilities_one_valid > threshold
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    print('Оптимальный порог:', best_threshold)
    print('F1:', best_f1, '| AUC-ROC:', auc_roc)
    print()


print('Логистическая регрессия')
model_1 = LogisticRegression(random_state=12345, solver='liblinear')
f1, auc_roc = obuchenie(model_1, features_train, target_train)
porog(model_1)
print()


print('Решающее дерево')
best_depth = 0
best_f1 = 0
best_model_2 = None
for depth in range(1, 20):
    model_2 = DecisionTreeClassifier(max_depth = depth, random_state = 12345)
    f1, auc_roc = obuchenie(model_2, features_train, target_train)
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth
        best_model_2 = model_2
print('Оптимальная глубина:', best_depth)
porog(best_model_2)
print()



print('Случайный лес')
best_est = 0
best_depth = 0
best_f1 = 0
best_model_3 = None
for est in range(10, 101, 10):
    for depth in range(1, 13):
        model_3 = RandomForestClassifier(random_state = 12345, n_estimators = est, max_depth=depth)
        f1, auc_roc = obuchenie(model_3, features_train, target_train)
        if f1 > best_f1:
            best_f1 = f1
            best_est = est
            best_depth = depth
            best_model_3 = model_3
print('Оптимальное количество деревьев:', best_est)
print('Оптимальная глубина:', best_depth)
porog(best_model_3)
print()

Логистическая регрессия
Оптимальный порог: 0.28
F1: 0.5110132158590308 | AUC-ROC: 0.774481723634266


Решающее дерево
Оптимальная глубина: 8
Оптимальный порог: 0.48
F1: 0.5868945868945868 | AUC-ROC: 0.8018072001122848


Случайный лес
Оптимальное количество деревьев: 100
Оптимальная глубина: 12
Оптимальный порог: 0.34
F1: 0.6378653113087674 | AUC-ROC: 0.8666864090592904


CPU times: user 46.3 s, sys: 320 ms, total: 46.6 s
Wall time: 47.5 s


Для каждого варианта работы с дисбалансом мы обучали новую модель и искали под нее лучшие гиперпараметры. Результат выше.  <br>
    
Стоит сказать, что лучшая модель - это модель "случайного леса", полученная при взвешивании классов. <br>
Метрика F1 = 0.64 | AUC-ROC = 0.87
    
Отправляемся тестировать.


<a id='4'></a>

# 4. Тестирование модели

### 1ый способ

In [16]:
# Возьмем лучшую модель с ее гиперпараметрами и обучим на обучающей выборке
model_3 = RandomForestClassifier(random_state = 12345, n_estimators = 60, max_depth = 7, class_weight = 'balanced')
model_3.fit(features_train, target_train)

# F1-мера лучшей модели на валидационных данных
predictid_valid = model_3.predict(features_valid)
f1 = f1_score(target_valid, predictid_valid)
probabilities_valid = model_3.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('Валидационная выборка')
print('F1:', f1, '| AUC-ROC:', auc_roc)
print()

# F1-мера лучшей модели на тестовых данных
predicted_test = model_3.predict(features_test)
f1 = f1_score(target_test, predicted_test)
probabilities_test = model_3.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('Тестовая выборка')
print('F1:', f1, '| AUC-ROC:', auc_roc)

Валидационная выборка
F1: 0.6436285097192225 | AUC-ROC: 0.8665661038542395

Тестовая выборка
F1: 0.6224598930481283 | AUC-ROC: 0.8705346332464976


### 2ой способ

In [17]:
# Объединим обучающие и валидационные выборки
features_train_valid = pd.concat([features_train, features_valid])
target_train_valid = pd.concat([target_train, target_valid])

# Возьмем лучшую модель с ее гиперпараметрами и обучим на объединенной выборке
model_3 = RandomForestClassifier(random_state = 12345, n_estimators = 60, max_depth = 7, class_weight = 'balanced')
model_3.fit(features_train_valid, target_train_valid)

# F1-мера лучшей модели на тестовых данных
predicted = model_3.predict(features_test)
f1 = f1_score(target_test, predicted)
probabilities_test = model_3.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('Тестовая выборка')
print('F1:', f1, '| AUC-ROC:', auc_roc)

Тестовая выборка
F1: 0.6335797254487857 | AUC-ROC: 0.8722127366195164


<a id='5'></a>

# 5. Общий вывод

Модель прогнозирования ухода клиента из банка построена, достигнуты хорошие значения метрики. Осталось подобрать к каждому свой подход, чтобы удержать клиента в финансовой структуре. 
    
Метрика AUC-ROC тоже не осталась без исследования. По цифрам видно, как она зависит от F1: если представить график, по вертикальной оси которого будет метрика AUC-ROC, а по горизонтальной F1, то эта зависимость будет отрисовываться по параболе. При малых значениях метрики F1 - значения метрики AUC-ROC будут высокими, и наоборот, при больших, значениях метрики F1 - значения метрики AUC-ROC будут высокими, однако в центре значений F1 существует некий порог, при котором значения метрики AUC-ROC будут низкими. 